In [ ]:
import requests
import json

# Remplacez par votre clé API OpenWeatherMap
api_key = "7d0eac5289d287cefce2099a08453213"

# Liste de villes
cities = ["Paris", "Toulouse", "Montpellier", "Nantes", "Bordeaux", "Cahors"]

# Boucle sur chaque ville pour faire une requête séparée
for city_name in cities:
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}"
    
    # Effectuer une requête à l'API
    response = requests.get(base_url)
    
    if response.status_code == 200:
        data = response.json()
        
        # Extraire les informations pertinentes
        city_info = {
            "city": data['name'],
            "country": data['sys']['country'],
            "temperature": data['main']['temp'],
            "humidity": data['main']['humidity'],
            "pressure": data['main']['pressure'],
            "weather": data['weather'][0]['description'],
            "wind_speed": data['wind']['speed'],
            "timestamp": data['dt']
        }
        
        # Afficher les données extraites
        print(json.dumps(city_info, indent=4))
    else:
        print(f"Erreur lors de la requête pour {city_name}: {response.status_code}")



In [ ]:
from kafka import KafkaProducer
# Configurer le producteur Kafka
producer = KafkaProducer(
    bootstrap_servers='kafka:9092',  # Adresse du serveur Kafka
    value_serializer=lambda v: json.dumps(v).encode('utf-8')  # Sérialiser les données en JSON
)


# Nom du topic Kafka
topic_name = 'weather_data'

# Boucle sur chaque ville pour faire une requête séparée
for city_name in cities:
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}"
    
    # Effectuer une requête à l'API
    response = requests.get(base_url)
    
    if response.status_code == 200:
        data = response.json()
        
        # Extraire les informations pertinentes
        city_info = {
            "city": data['name'],
            "country": data['sys']['country'],
            "temperature": data['main']['temp'],
            "humidity": data['main']['humidity'],
            "pressure": data['main']['pressure'],
            "weather": data['weather'][0]['description'],
            "wind_speed": data['wind']['speed'],
            "timestamp": data['dt']
        }
        
        # Envoyer les données au topic Kafka
        producer.send(topic_name, value=city_info)
        print(f"Données envoyées à Kafka pour {city_name}: {json.dumps(city_info, indent=4)}")
    else:
        print(f"Erreur lors de la requête pour {city_name}: {response.status_code}")

# S'assurer que toutes les données sont envoyées
producer.flush()

In [ ]:
from kafka import KafkaConsumer
from pymongo import MongoClient
import json

# Configuration du consommateur Kafka
consumer = KafkaConsumer(
    'weather_data',
    bootstrap_servers='kafka:9092',
    group_id='weather-group',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

# Connexion à MongoDB
mongo_client = MongoClient('https://localhost:27017')
db = mongo_client.weather_db
collection = db.weather_data

# Consommer les messages Kafka et les insérer dans MongoDB
for message in consumer:
    data = message.value
    collection.insert_one(data)
    print(f"Données insérées dans MongoDB: {json.dumps(data, indent=4)}")
